# Library Usage in Seattle, 2005-2020

## Data Cleaning

The data is the [Checkouts by Title (Physical Items)](https://data.seattle.gov/Community/Checkouts-By-Title-Physical-Items-/5src-czff) dataset from [Seattle Open Data](https://data.seattle.gov/) and was downloaded on December 15, 2020.

This notebook is designed to load a downloaded CSV file, merge it with item-specific information, convert it to a time-series-ready DataFrame, and save that as a compressed pickle file.

*Note: This dataset is updated weekly; the more data, the longer the load times will be.*

In the future, it may be a good idea to look into adding [API](https://dev.socrata.com/foundry/data.seattle.gov/5src-czff) calls into the pipeline, so as to quickly and easily add on the additional weekly data.

*Note: Any cell that uses the built-in magic command* `%%time` *takes a significant (or at least not insignificant) time to run.*

### Import required libraries

In [1]:
# standard dataframe packages
import pandas as pd
import numpy as np

# saving packages
import pickle
import gzip

from functions.data_cleaning import status_update

### Load checkout data

[[go back to the top](#Library-Usage-in-Seattle,-2005-2020)]

Since the data set is so large, I'll specify only the columns that I want in the DataFrame. This will effectively drop the following columns:
- `ID`
- `CheckoutYear`
- `CallNumber`
- `BibNumber`
- `ItemBarcode`
- `ItemType`
    
I want to note that the `ItemType` and `Collection` columns are very similar, but the code in the `Collection` column contains more information within the `category_group` column that I add onto the DataFrame using the `data_dictionary.csv` file ([see below](#Load-other-info-from-data-dictionary-and-merge-onto-checkouts-dataset)). More specifically, the `ItemType` code yields mostly "Miscellaneous" results, whereas the `Collection` code yields differentiates between "Fiction" and "Nonfiction", among others. This could be useful information later on, so I found it best to drop the `ItemType` column.

Note: In the future, I may consider using this [dataset](https://data.seattle.gov/Community/Library-Collection-Inventory/6vkj-f5xf) to add on branch information (i.e. which branch an item was checked out from), although this data is rather limited (due to privacy concerns) and incomplete (only appears to be collected beginning in 2017). In order to do that I would need to use the `BibNumber` column.

#### ⏰ Cell below takes ~14.5 minutes to run. ⏰

In [2]:
%%time

# columns to load
usecols = ['Collection', 'ItemTitle', 'Subjects', 'CheckoutDateTime']

# load data
df = pd.read_csv('data/Checkouts_By_Title__Physical_Items_.csv',
#                  nrows=10000000,
                 usecols=usecols)

# rename columns to my preferred format
df.columns = ['collection', 'title', 'subjects', 'date']

CPU times: user 4min 20s, sys: 4min 39s, total: 9min
Wall time: 14min 1s


*Note: While it only took about 25 seconds to load 10 million rows, it takes about 25 minutes to load 106.5 million rows with the same number (5) of columns. In my latest update, I brought it down to 4 columns, which decreased load time to 15.5 minutes.*

In [3]:
# check shape
df.shape

(106534901, 4)

In [4]:
# take a look
df.head()

,collection,title,subjects,date
0,nadvd,Firewall,"Kidnapping Drama, Video recordings for the hea...",02/13/2008 07:38:00 PM
1,nanf,best baby shower book a complete guide for par...,Showers Parties,07/23/2008 02:53:00 PM
2,nyfic,Uglies,"Fantasy, Teenage girls Fiction, Beauty Persona...",12/23/2009 04:20:00 PM
3,napar,doula guide to birth secrets every pregnant wo...,"Doulas, Childbirth",11/16/2010 12:04:00 PM
4,canf,Salmon a cookbook,Cookery Salmon,04/26/2009 01:29:00 PM


#### ⏰ Cell below takes ~3 minutes to run. ⏰

In [5]:
%%time

# check for nan values
df.isna().sum()

CPU times: user 33.6 s, sys: 1min 9s, total: 1min 43s
Wall time: 3min 19s


collection          0
title          900912
subjects      1649522
date                0
dtype: int64

*NOTE: Even checking for NaN values takes a significant amount of time with this many rows.*

The most important columns (`collection` and `date`) have no NaN values.

In [6]:
# check datatypes
df.dtypes

collection    object
title         object
subjects      object
date          object
dtype: object

### Convert `date` column to datetime

[[go back to the top](#Library-Usage-in-Seattle,-2005-2020)]

In [7]:
# look at an example before conversion
df.loc[0, 'date']

'02/13/2008 07:38:00 PM'

In [8]:
# specify the format
dt_format = '%m/%d/%Y %I:%M:%S %p'

#### ⏰ Cell below takes ~7 minutes to run. ⏰

In [9]:
%%time

# convert to datetime, dropping the hour-minute-second stamp using the `dt.date` attribute
df['date'] = pd.to_datetime(df.date, format=dt_format).dt.date

# confirm it worked
df.loc[0, 'date']

CPU times: user 6min 27s, sys: 15.6 s, total: 6min 43s
Wall time: 6min 46s


datetime.date(2008, 2, 13)

### Load other info from data dictionary and merge onto checkouts dataset

[[go back to the top](#Library-Usage-in-Seattle,-2005-2020)]

In [10]:
# load data
dd = pd.read_csv('data/data_dictionary.csv')

# rename columns to my preferred format
dd.columns = ['code', 'description', 'code_type', 'format_group', 'format_subgroup', 
              'category_group', 'category_subgroup', 'age_group']

# take a look
dd.head()

,code,description,code_type,format_group,format_subgroup,category_group,category_subgroup,age_group
0,cazover,CA7-zine collection oversize,ItemCollection,Print,Book,Periodical,NaN,Adult
1,caziner,CA7-zine collection reference,ItemCollection,Print,Book,Periodical,NaN,Adult
2,cazval,CA7-zine collection valuable mat.,ItemCollection,Print,Book,Periodical,NaN,Adult
3,nga,Northgate Branch,Location,NaN,NaN,NaN,NaN,NaN
4,hip,High Point Branch,Location,NaN,NaN,NaN,NaN,NaN


In [11]:
# check shape
dd.shape

(580, 8)

In [12]:
# check datatypes
dd.dtypes

code                 object
description          object
code_type            object
format_group         object
format_subgroup      object
category_group       object
category_subgroup    object
age_group            object
dtype: object

Since I will only be using information from codes whose type is "ItemCollection", I'll subset the data dictionary down to just those rows.

In [13]:
# subset to only collection codes
dd = dd[dd.code_type == 'ItemCollection']

In [14]:
# check for nan values
dd.isna().sum()

code                   0
description            0
code_type              0
format_group           0
format_subgroup       28
category_group         2
category_subgroup    391
age_group              0
dtype: int64

Again, with the size of the eventual DataFrame in mind, I want to drop any unnecessary columns before merging, so I'll drop the following columns:
- `description`, since that is superfluous information for this project
- `code_type`, since that is superfluous information
- `category_subgroup`, since that is mostly NaN values

In [15]:
# drop columns
dd.drop(columns=['description', 'code_type', 'category_subgroup'], inplace=True)

In [16]:
# list of columns to convert
to_convert = ['format_group', 'format_subgroup', 'category_group', 'age_group']

# convert to category datatype
dd[to_convert] = dd[to_convert].apply(pd.Categorical)

In [17]:
# confirm new datatypes
dd.dtypes

code                 object
format_group       category
format_subgroup    category
category_group     category
age_group          category
dtype: object

#### ⏰ Cell below takes ~4 minutes to run. ⏰

In [18]:
%%time

# merge checkouts dataframe with info from data dictionary
df_merged = df.merge(dd, left_on='collection', right_on='code')

# take a look
df_merged.head()

CPU times: user 1min, sys: 1min 50s, total: 2min 51s
Wall time: 4min 23s


,collection,title,subjects,date,code,format_group,format_subgroup,category_group,age_group
0,nadvd,Firewall,"Kidnapping Drama, Video recordings for the hea...",2008-02-13,nadvd,Media,Video Disc,Fiction,Adult
1,nadvd,Marley me,"Comedy films, Married people Drama, Philadelph...",2009-07-03,nadvd,Media,Video Disc,Fiction,Adult
2,nadvd,Six feet under The complete fourth season,"Video recordings for the hearing impaired, Pro...",2008-10-26,nadvd,Media,Video Disc,Fiction,Adult
3,nadvd,Doctor Who The next doctor,"London England Drama, Doctor Who Fictitious ch...",2010-11-10,nadvd,Media,Video Disc,Fiction,Adult
4,nadvd,School ties,"Antisemitism Drama, Video recordings for the h...",2008-12-28,nadvd,Media,Video Disc,Fiction,Adult


### Drop unnecessary columns

I can now drop the `collection` and `code` columns, since those are no longer necessary.

*NOTE: Using the Pandas method `.drop()` was taking well over an hour, so I'm going to try to subset it below, to see if that works any faster.*

#### ⏰ Cell below takes ~38.5 minutes to run. ⏰

In [19]:
%%time

# drop columns
df_merged.drop(columns=['collection', 'code'], inplace=True)

CPU times: user 2min 43s, sys: 18min 6s, total: 20min 50s
Wall time: 44min 42s


### Set `date` column as index

[[go back to the top](#Library-Usage-in-Seattle,-2005-2020)]

I've commented out the below code because now *this* has begun to consistently crash the kernel or zsh shell.

After thinking more about it, I believe I will end up grouping by the date to get raw numbers for each day (not only total checkouts, but total print checkouts and fiction checkouts, etc.), which can be done before setting the `date` column as the index.

In [20]:
# %%time

# # set `date` column as index and sort by index
# df_merged = df_merged.set_index('date').sort_index()

# # take a look
# df_merged.head()

In [21]:
# check shape
df_merged.shape

(106503843, 7)

In [22]:
# check datatypes
df_merged.dtypes

title                object
subjects             object
date                 object
format_group       category
format_subgroup    category
category_group     category
age_group          category
dtype: object

NOTE: I may be able to drop even more columns (thinking especially of `title` and `subjects`), since I'll mostly be looking at sheer numbers of items checked out each day. I'll keep them in for now in case they end up being useful for EDA.

### 💾 Save

[[go back to the top](#Library-Usage-in-Seattle,-2005-2020)]

Due to some several kernel and zsh shell crashes, I'm going to try to save the DataFrame in batches of 10 million rows.

*NOTE: Save time for 10 million rows takes about 5 minutes and the file size is 290MB. Increasing to 20 million rows seemed to increase save time considerably, and so was interrupted before completing.*

⏰ The next cell will most likely take ~10 hours to run. ⏰

In [23]:
# %%time

# # loop through index and multiples of 10 million
# for ind, i in enumerate(range(0, 110000000, 10000000), 1):
    
#     # save (via compressed pickle) a dataframe of 10 million rows, use index for unique file names
#     df_merged.iloc[i:i+10000000].to_pickle(f'data/seattle_lib_{ind}.pkl', compression='gzip')
    
#     # print status/time
#     status_update(f'File {ind} out of 11 saved successfully')

If anything gets stuck or your computer crashes, uncomment the code below, replace `n` with the part to start the new loop with, and run the cell.

In [24]:
# %%time

# n = ____

# # loop through index and multiples of 10 million
# for ind, i in enumerate(range((n-1)*10000000, 110000000, 10000000), n):
    
#     # save (via compressed pickle) a dataframe of 10 million rows, use index for unique file names
#     df_merged.iloc[i:i+10000000].to_pickle(f'data/seattle_lib_{ind}.pkl', compression='gzip')
    
#     # print status/time
#     status_update(f'File {ind} out of 11 saved successfully')

Current time = 00:53:04
-----------------------
File 5 out of 11 saved successfully

Current time = 01:18:18
-----------------------
File 6 out of 11 saved successfully

Current time = 02:35:13
-----------------------
File 7 out of 11 saved successfully

Current time = 04:42:51
-----------------------
File 8 out of 11 saved successfully

Current time = 06:06:44
-----------------------
File 9 out of 11 saved successfully

Current time = 06:43:35
-----------------------
File 10 out of 11 saved successfully

CPU times: user 28min 7s, sys: 2h 17min 27s, total: 2h 45min 35s
Wall time: 6h 10min 4s


### 💾 Load

[[go back to the top](#Library-Usage-in-Seattle,-2005-2020)]

In [110]:
%%time

# print status/time
status_update('Begin load...')

# load first part
df = pd.read_pickle('data/seattle_lib_1.pkl', compression='gzip')

# print status/time
status_update('File 1 loaded successfully.')

# iterate through 2-11
for i in range(2, 12):

    # load parts 2-11
    to_add = pd.read_pickle(f'data/seattle_lib_{i}.pkl', compression='gzip')
    
    # print status/time
    status_update(f'File {i} loaded successfully.')

    # combine with previous part
    df = pd.concat([df, to_add], ignore_index=True)
    
    # print status/time
    status_update(f'Concatenation successful. DataFrame consists of files 1-{i}.')
    
# print status/time
status_update('Load complete!')

Current time = 16:59:31
-----------------------
Begin load...

Current time = 16:59:50
-----------------------
File 1 loaded successfully.

Current time = 17:00:13
-----------------------
File 2 loaded successfully.

Current time = 17:00:18
-----------------------
Concatenation successful. DataFrame consists of files 1-2.

Current time = 17:00:35
-----------------------
File 3 loaded successfully.

Current time = 17:00:37
-----------------------
Concatenation successful. DataFrame consists of files 1-3.

Current time = 17:00:50
-----------------------
File 4 loaded successfully.

Current time = 17:00:52
-----------------------
Concatenation successful. DataFrame consists of files 1-4.

Current time = 17:01:09
-----------------------
File 5 loaded successfully.

Current time = 17:01:41
-----------------------
Concatenation successful. DataFrame consists of files 1-5.

Current time = 17:02:01
-----------------------
File 6 loaded successfully.

Current time = 17:02:55
-------------------

### Checking for duplicates and assumption of data integrity

[[go back to the top](#Library-Usage-in-Seattle,-2005-2020)]

The issue of checking for duplicates with this dataset is that duplicates are acceptable! It is very likely that the same item is checked out from either the same or different branches on a single day. Multiple copies of a book, for example, can be stored in one branch or across several branches.

NOTE: More investigation on the uniqueness of an item's call number could potentially solve this and allow me to check for actual duplicate rows. For the time being, I will assume the data is almost entirely, if not entirely, accurate.

Checking for duplicates can be done by uncommenting and running the code below, although it may take quite awhile to run.

In [94]:
# %%time

# df_merged[df_merged.duplicated(keep=False)]

### Investigate categorical columns

[[go back to the top](#Library-Usage-in-Seattle,-2005-2020)]

As shown above, the data has no NaN values for the all important `date` column (the count of which will be the target for my eventual time series models), which is great news.

In terms of the categorical columns, I'll keep NaN values as they are, which means they simply won't factor into the category counts, but the items *will* still be counted in terms of how many items were checked out in a day.

In [95]:
# format_group breakdown
df.format_group.value_counts()

Print         59685137
Media         46618209
Other           200478
Equipment           18
Electronic           1
Name: format_group, dtype: int64

Since `Equipment` and `Electronic` have so few observations, I'll lump those in with `Other` in the next section.

In [96]:
# format_subgroup breakdown
df.format_subgroup.value_counts()

Book              59486648
Video Disc        30287406
Audio Disc        11238813
Audiobook Disc     2695078
Video Tape         1474457
Kit                 626008
Audiobook Tape      240328
Music Score         130486
Audio Tape           45946
Folder               23900
Data Disc             9886
Periodical             623
Document               471
Art                    129
Film                    81
Name: format_subgroup, dtype: int64

I think I can keep all these as is. They all seem to contain a fair amount of information and should interesting as part of a dashboard or in EDA in general.

In [97]:
# category_group breakdown
df.category_group.value_counts()

Fiction              65292861
Nonfiction           37539002
Miscellaneous         1734282
Language              1679452
Interlibrary Loan      192959
Reference               57420
On Order                 7422
Temporary                  40
WTBBL                      26
Periodical                  0
Name: category_group, dtype: int64

Some of these--`Miscellaneous`, `On Order`, `Temporary`--can be simplified into an `Other` category, since their current category doesn't provide any valuable information in terms of what the actual item is. I was originally considering including `Interlibrary Loan` in the `Other` category as well, but it may be interesting to see the numbers behind activity between the branches, so I'll leave it in for now.

Based on some research, `WTBBL` stands for "Washington Talking Book Library" and includes materials for folks with visual impairments. I was interested in looking into this, but since the numbers are so low, I think I'll also group that into the `Other` category, as I assume this `category_group` value deal with *equipment* that can be rented.

I will also convert `Periodical` to other; even though the count for it is 0, I assume this may relate to to items that have a `format_subgroup` value but no `category_group` value.

In [98]:
# category_group breakdown
df.age_group.value_counts()

Adult       71587854
Juvenile    31006173
Teen         3909816
Name: age_group, dtype: int64

Everything looks good here!

### Transform values in `format_group` and `category_group`

[[go back to the top](#Library-Usage-in-Seattle,-2005-2020)]

In [ ]:
# convert_values = ['Miscellaneous', 'On Order', 'Temporary', 'WTBBL', 'Periodical']

# test['category_group'] = np.where(test.category_group in convert_values, 'Other')

In [111]:
%%time

# values to convert
convert_values = ['Equipment', 'Electronic']

# custom function to transform
df['format_group'] = transform_category(df, 'format_group', convert_values, 'Other')

# confirm
df.format_group.unique()

CPU times: user 6.72 s, sys: 2.7 s, total: 9.42 s
Wall time: 9.8 s


[Media, Print, Other]
Categories (3, object): [Media, Print, Other]

In [112]:
%%time

# values to convert
convert_values = ['Miscellaneous', 'On Order', 'Temporary', 'WTBBL', 'Periodical']

# custom function to transform
df['category_group'] = transform_category(df, 'category_group', convert_values, 'Other')

# confirm
df.category_group.unique()

CPU times: user 8.36 s, sys: 2.23 s, total: 10.6 s
Wall time: 10.7 s


[Fiction, Nonfiction, Language, Other, Interlibrary Loan, Reference, NaN]
Categories (6, object): [Fiction, Nonfiction, Language, Other, Interlibrary Loan, Reference]

### Dummy and count the necessary columns

In [113]:
%%time

# columns to dummy
dummy_cols = ['format_group', 'format_subgroup', 'category_group', 'age_group']

# dummy the columns
dummy_df = pd.get_dummies(df[dummy_cols], prefix=dummy_cols)

# take a look
dummy_df.head()

CPU times: user 9.28 s, sys: 6.04 s, total: 15.3 s
Wall time: 15.9 s


,format_group_Media,format_group_Other,format_group_Print,format_subgroup_Art,format_subgroup_Audio Disc,format_subgroup_Audio Tape,format_subgroup_Audiobook Disc,format_subgroup_Audiobook Tape,format_subgroup_Book,format_subgroup_Data Disc,...,format_subgroup_Video Tape,category_group_Fiction,category_group_Interlibrary Loan,category_group_Language,category_group_Nonfiction,category_group_Other,category_group_Reference,age_group_Adult,age_group_Juvenile,age_group_Teen
0,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0


In [114]:
%%time

# combine with date column
df_counts = pd.concat([df.date, dummy_df], axis=1)

# take a look
df_counts.head()

CPU times: user 6.31 s, sys: 12.3 s, total: 18.6 s
Wall time: 22.1 s


,date,format_group_Media,format_group_Other,format_group_Print,format_subgroup_Art,format_subgroup_Audio Disc,format_subgroup_Audio Tape,format_subgroup_Audiobook Disc,format_subgroup_Audiobook Tape,format_subgroup_Book,...,format_subgroup_Video Tape,category_group_Fiction,category_group_Interlibrary Loan,category_group_Language,category_group_Nonfiction,category_group_Other,category_group_Reference,age_group_Adult,age_group_Juvenile,age_group_Teen
0,2008-02-13,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
1,2009-07-03,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
2,2008-10-26,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
3,2010-11-10,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
4,2008-12-28,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0


In [ ]:
%%time

# group by date and get category total for each column
df_counts = df_counts.groupby('date').agg('sum')

# take a look
df_counts.head()

In [92]:
%%time

# combine with total checkouts per day
df_counts = pd.concat([df.groupby('date').size(), df_counts], axis=1)

# rename target column
df_counts.columns[0] = 'total_checkouts'

# take a look
df_counts.head()

CPU times: user 19.4 s, sys: 9.07 s, total: 28.5 s
Wall time: 33.7 s


,0,format_group_Electronic,format_group_Equipment,format_group_Media,format_group_Other,format_group_Print,format_subgroup_Fiction,format_subgroup_Interlibrary Loan,format_subgroup_Language,format_subgroup_Nonfiction,...,category_group_Miscellaneous,category_group_Nonfiction,category_group_On Order,category_group_Periodical,category_group_Reference,category_group_Temporary,category_group_WTBBL,age_group_Adult,age_group_Juvenile,age_group_Teen
date,,,,,,,,,,,,,,,,,,,,,
2005-04-13,16471,0.0,1.0,6397.0,32.0,10041.0,8189.0,32.0,370.0,6719.0,...,1143.0,6719.0,0.0,0.0,18.0,0.0,0.0,11257.0,4613.0,601.0
2005-04-14,10358,0.0,1.0,4015.0,75.0,6267.0,5276.0,73.0,272.0,4104.0,...,619.0,4104.0,2.0,0.0,12.0,0.0,0.0,6726.0,3381.0,251.0
2005-04-15,12896,0.0,0.0,5351.0,51.0,7494.0,6357.0,50.0,302.0,5166.0,...,1013.0,5166.0,1.0,0.0,7.0,0.0,0.0,8795.0,3747.0,354.0
2005-04-16,1358,0.0,0.0,552.0,0.0,806.0,567.0,0.0,29.0,666.0,...,95.0,666.0,0.0,0.0,1.0,0.0,0.0,950.0,367.0,41.0
2005-04-17,4555,0.0,0.0,1555.0,8.0,2992.0,2017.0,8.0,177.0,2145.0,...,203.0,2145.0,0.0,0.0,5.0,0.0,0.0,3035.0,1349.0,171.0


## GRAVEYARD

[[go back to the top](#Library-Usage-in-Seattle,-2005-2020)]

In [34]:
%%time

cols = ['date', 'format_group', 'format_subgroup', 'category_group', 'age_group']

test[cols].groupby(cols).size()

CPU times: user 3min 15s, sys: 1min 56s, total: 5min 12s
Wall time: 5min 42s


date        format_group  format_subgroup  category_group     age_group
2008-01-02  Electronic    Art              Fiction            Adult        0
                                                              Juvenile     0
                                                              Teen         0
                                           Interlibrary Loan  Adult        0
                                                              Juvenile     0
                                                                          ..
2019-07-18  Print         Video Tape       Temporary          Juvenile     0
                                                              Teen         0
                                           WTBBL              Adult        0
                                                              Juvenile     0
                                                              Teen         0
Length: 9139500, dtype: int64

In [ ]:
# %%time

# # list of columns to keep
# keep_cols = ['title', 'subjects', 'date', 'format_group', 'format_subgroup',
#              'category_group', 'age_group']

# # drop columns
# df_merged = df_merged[keep_cols]

In [ ]:
dd[dd.code_type == 'ItemType']

In [ ]:
dd[dd.code == 'nadvd']

In [ ]:
dd[dd.code == 'acdvd']

In [ ]:
dd.code_type.unique()

In [ ]:
dd_item = dd[dd.code_type == 'ItemType'][['code', 'description', 'format_group', 'format_subgroup', 'category_group', 
             'category_subgroup', 'age_group']]

dd_item.head()

In [ ]:
dd_item2 = dd[dd.code_type == 'ItemCollection'][['code', 'description', 'format_group', 'format_subgroup', 'category_group', 
             'category_subgroup', 'age_group']]

dd_item2.head()

In [ ]:
sorted(df.item_type.unique())

In [ ]:
dd_loc = dd[dd.code_type == 'Location'][['code', 'description']]

dd_loc.head()

In [ ]:
test = df.merge(dd_item, left_on='item_type', right_on='code')
# test = test.merge(dd_loc, left_on='collection', right_on='code')

test.head()

In [ ]:
test.isna().sum()

In [ ]:
test.format_group.value_counts()

In [ ]:
test.collection.unique()

In [ ]:
test.shape

In [ ]:
test2 = df.merge(dd_item2, left_on='Collection', right_on='code')

test2.head()

In [ ]:
test.groupby('format_group').category_group.value_counts()

In [ ]:
test2.groupby('format_group').category_group.value_counts()

In [ ]:
dd[dd.code == 'nybot']

In [ ]:
sorted(df.collection.unique())

In [ ]:
dd_loc.code.unique()

In [ ]:
[cod for cod in df.collection.unique() if cod in dd_loc.code.unique()]

#### NOTE: Using the code below on 10 million rows is almost 39% faster and results in a saved file that is *nearly the same size as the original CSV file* (file is 23.22GB!).

In [ ]:
# %%time

# df_merged.to_hdf('data/seattle_lib_temp_ten_mil__alt.hdf', 'mydata', format='table', mode='w')

In [ ]:
df.title.value_counts().head(10)

In [ ]:
df[df.title=='reader'].head()